# When to Use Extended Thinking
The decision is straightforward: use your prompt evaluations. Run your prompts without thinking first, and if the accuracy isn't meeting your requirements after you've already optimized your prompt, then consider enabling extended thinking. It's a tool for when standard prompting isn't quite getting you there.

# Redacted Thinking
Sometimes you'll receive a redacted thinking block instead of readable reasoning text:

This happens when Claude's thinking process gets flagged by internal safety systems. The redacted content contains the actual thinking in encrypted form, allowing you to pass the complete message back to Claude in future conversations without losing context.

# dependencies

In [1]:
from anthropic import Anthropic
from anthropic.types import Message

from dotenv import load_dotenv
import os   
load_dotenv()

import logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
    )
logger = logging.getLogger(__name__)

try:
    client = Anthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )
    logger.info("Anthropic client initialized successfully.")
except Exception as e:
    logger.error(f"Failed to initialize Anthropic client: {e}")
    raise e

2025-08-10 19:04:41 - __main__ - INFO - Anthropic client initialized successfully.


# global config

In [2]:
MODEL="claude-3-7-sonnet-20250219"
TEMPERATURE=1.0

# helper functions

In [ ]:
def add_user_message(messages, message_content):
    if isinstance(message_content, Message):
        user_message = {
            "role": "user",
            "content": message_content
        }
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": str(message_content)}]}

    messages.append(user_message)


def add_assistant_message(messages, message_content):
    if isinstance(message_content, Message):
        assistant_message = {
            "role": "assistant",
            "content": message_content
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": [{"type": "text", "text": message_content}]
        }

    messages.append(assistant_message)

def chat(messages, model=MODEL, temperature=TEMPERATURE, system=None, stop_sequences=None, thinking=False, thinking_budget=1024, tools=None):
    try:
        params = {
            "model": model,
            "max_tokens": 4000,
            "messages": messages,
            "temperature": temperature,
            "stop_sequences": stop_sequences,
        }
        
        if thinking:
            params["thinking"] = {
                "type": "enabled",
                "budget_tokens": thinking_budget
            }
        
        if tools:
            params["tools"] = tools

        if system:
            params["system"] = system

        return client.messages.create(**params)
    
    except Exception as e:
        logger.error(f"Chat failed: {e}")
        raise e
    
def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

# main function

In [4]:
def main():
    messages = []

    add_user_message(messages, "What is inflation?")
    
    response = chat(messages, thinking=True)

    return response

In [5]:
results = main()
print("results:", results)

2025-08-10 19:04:51 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


results: Message(id='msg_019794TUBtKMaRJLMucknD8Q', content=[ThinkingBlock(signature='ErUBCkYIBhgCIkCvXvvTVsTV4OHb4k3Xf8PXXd44n4CpZ3ino8dqUio3bUvxSdbFP8FeeKgPSLIVM16rHJHCw8HzRAUuwHNa8LTOEgy1pH+sGV47PB/92+QaDB31miEV867/GoAtyCIwB5oYAQDcjCJdByXWP0Vu/RNTiBDiFkwWX46xXp72zQjTkWZNUPwVxBHDro4isCiYKh1JKV2lIjp7Y5+kzVlO2qcaN3UDCilJ4+h/HF0GNRgC', thinking='Inflation is a term from economics that refers to the general increase in prices of goods and services over time in an economy, which results in a decline in the purchasing power of money. \n\nThere are several types of inflation:\n1. Demand-pull inflation: Occurs when demand for goods and services exceeds supply.\n2. Cost-push inflation: Happens when the cost of production increases, pushing up prices.\n3. Built-in inflation: Results from adaptive expectations, often linked to the price/wage spiral.\n\nInflation is typically measured by the Consumer Price Index (CPI) or other price indices. Central banks often target a low, stable inflation rat

In [ ]:
print(f"thinking signature: \n{results.content[0].signature}")
print(f"\nthinking content: \n{results.content[0].thinking}")

thinking signature: 
ErUBCkYIBhgCIkCvXvvTVsTV4OHb4k3Xf8PXXd44n4CpZ3ino8dqUio3bUvxSdbFP8FeeKgPSLIVM16rHJHCw8HzRAUuwHNa8LTOEgy1pH+sGV47PB/92+QaDB31miEV867/GoAtyCIwB5oYAQDcjCJdByXWP0Vu/RNTiBDiFkwWX46xXp72zQjTkWZNUPwVxBHDro4isCiYKh1JKV2lIjp7Y5+kzVlO2qcaN3UDCilJ4+h/HF0GNRgC

thinking content: 
Inflation is a term from economics that refers to the general increase in prices of goods and services over time in an economy, which results in a decline in the purchasing power of money. 

There are several types of inflation:
1. Demand-pull inflation: Occurs when demand for goods and services exceeds supply.
2. Cost-push inflation: Happens when the cost of production increases, pushing up prices.
3. Built-in inflation: Results from adaptive expectations, often linked to the price/wage spiral.

Inflation is typically measured by the Consumer Price Index (CPI) or other price indices. Central banks often target a low, stable inflation rate (commonly around 2%) as part of their monetary policy.

Causes

In [ ]:
print(f"assistant text response: \n{results.content[1].text}")

assistant text response: 
# Inflation

Inflation is the general increase in prices of goods and services in an economy over time, which reduces the purchasing power of money. When inflation occurs, each unit of currency buys fewer goods and services than it did previously.

## Key aspects of inflation:

- **Measurement**: Typically measured using indices like the Consumer Price Index (CPI), which tracks the average price changes of a basket of common goods and services
- **Causes**: Can result from increased demand exceeding supply, rising production costs, expansion of the money supply, or government policies
- **Effects**: Reduces purchasing power, can create economic uncertainty, affects interest rates, and may redistribute wealth

Most central banks aim to maintain a low, stable inflation rate (often around 2%) to support economic growth while preserving the value of money.

Is there a specific aspect of inflation you'd like to explore further?
